In [1]:
# from src.ExtremeWeatherBench import events
from src.ExtremeWeatherBench import config
from src.ExtremeWeatherBench import events
from src.ExtremeWeatherBench import evaluate
import importlib
importlib.reload(config)
import xarray as xr

In [2]:
event_list = [events.HeatWave]
heatwave_configuration = config.Config(
    event_types=event_list,
    )
default_forecast_config = config.ForecastSchemaConfig()

In [3]:
evaluate.evaluate(eval_config=heatwave_configuration)
# TODO need to figure how to trigger computation

/home/taylor/code/ExtremeWeatherBench/src/ExtremeWeatherBench/../../assets/data/events.yaml
HeatWave(heat_wave=[IndividualCase(id=1, start_date=datetime.date(2021, 6, 24), end_date=datetime.date(2021, 7, 2), location=Location(latitude=47.6062, longitude=-122.3321), bounding_box_km=500), IndividualCase(id=3, start_date=datetime.date(2021, 6, 24), end_date=datetime.date(2021, 7, 2), location=Location(latitude=34.0522, longitude=-118.2437), bounding_box_km=500)])


In [3]:
for n in heatwave_configuration.event_types[0].case_df.itertuples():
    print(type(n))

<class 'pandas.core.frame.Pandas'>


In [4]:
iteration_list = [n for n in heatwave_configuration.event_types[0].case_df.itertuples()]

iteration_list[0]


Pandas(Index=1, title='2021 Western North America Heat Wave', start_date=Timestamp('2021-06-24 00:00:00'), end_date=Timestamp('2021-07-02 00:00:00'), location=Location(latitude=47.6062, longitude=-122.3321), bounding_box_km=500, cross_listed=None)

In [5]:
era5 = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2",
    chunks='auto',
)

In [6]:
event_ds = era5.sel(time=slice(iteration_list[0].start_date, iteration_list[0].end_date))
event_ds

<xarray.Dataset> Size: 187GB
Dimensions:                                           (time: 193,
                                                       latitude: 721,
                                                       longitude: 1440,
                                                       level: 37)
Coordinates:
  * latitude                                          (latitude) float32 3kB ...
  * level                                             (level) int64 296B 1 .....
  * longitude                                         (longitude) float32 6kB ...
  * time                                              (time) datetime64[ns] 2kB ...
Data variables: (12/31)
    10m_u_component_of_wind                           (time, latitude, longitude) float32 802MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    10m_v_component_of_wind                           (time, latitude, longitude) float32 802MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    2m_temperature                                    (time, latitude, longitude) float32 802MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    geopotential                                      (time, level, latitude, longitude) float32 30GB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    ...                                                ...
    total_precipitation                               (time, latitude, longitude) float32 802MB dask.array<chunksize=(1, 721, 1440), meta=np.ndarray>
    type_of_high_vegetation                           (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    type_of_low_vegetation                            (latitude, longitude) float32 4MB dask.array<chunksize=(721, 1440), meta=np.ndarray>
    u_component_of_wind                               (time, level, latitude, longitude) float32 30GB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    v_component_of_wind                               (time, level, latitude, longitude) float32 30GB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>
    vertical_velocity                                 (time, level, latitude, longitude) float32 30GB dask.array<chunksize=(1, 37, 721, 1440), meta=np.ndarray>

In [7]:
import fsspec
fs = fsspec.filesystem('')
forecast_ds = evaluate._open_kerchunk_zarr_reference_jsons(fs.ls(heatwave_configuration.forecast_path), default_forecast_config)

In [8]:
forecast_ds

<xarray.Dataset> Size: 305GB
Dimensions:    (init_time: 26, fhour: 41, latitude: 721, longitude: 1440,
                level: 13, time: 91)
Coordinates:
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * level      (level) int32 52B 1000 925 850 700 600 500 ... 250 200 150 100 50
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 728B 2021-06-21 ... 2021-07-13T12:00:00
  * fhour      (fhour) float64 328B 0.0 6.0 12.0 18.0 ... 228.0 234.0 240.0
  * init_time  (init_time) datetime64[ns] 208B 2021-06-21 ... 2021-06-23
Data variables:
    msl        (init_time, fhour, latitude, longitude) float32 4GB dask.array<chunksize=(1, 28, 482, 960), meta=np.ndarray>
    q          (init_time, fhour, latitude, longitude, level) float32 58GB dask.array<chunksize=(1, 11, 181, 360, 3), meta=np.ndarray>
    t          (init_time, fhour, latitude, longitude, level) float32 58GB dask.array<chunksize=(1, 11, 181, 360, 3), meta=np.ndarray>
    t2         (init_time, fhour, latitude, longitude) float32 4GB dask.array<chunksize=(1, 28, 482, 960), meta=np.ndarray>
    u          (init_time, fhour, latitude, longitude, level) float32 58GB dask.array<chunksize=(1, 11, 181, 360, 3), meta=np.ndarray>
    u10        (init_time, fhour, latitude, longitude) float32 4GB dask.array<chunksize=(1, 28, 482, 960), meta=np.ndarray>
    v          (init_time, fhour, latitude, longitude, level) float32 58GB dask.array<chunksize=(1, 11, 181, 360, 3), meta=np.ndarray>
    v10        (init_time, fhour, latitude, longitude) float32 4GB dask.array<chunksize=(1, 28, 482, 960), meta=np.ndarray>
    z          (init_time, fhour, latitude, longitude, level) float32 58GB dask.array<chunksize=(1, 11, 181, 360, 3), meta=np.ndarray>
Attributes:
    Conventions:           CF-1.8
    creation_time:         2023-11-22T05:29:06
    first_forecast_hour:   6
    forecast_hour_step:    6
    initialization_model:  GFS
    initialization_time:   2021-06-21T00:00:00
    last_forecast_hour:    240
    model_name:            Pangu-Weather
    model_version:         v1
    version:               1_2023-10-14

In [14]:
events_file_path  = 'assets/data/events.yaml'
with open(events_file_path, 'r') as file:
    events_data = yaml.safe_load(file)['events']

events_data

{'heat_wave': [{'id': 1,
   'title': '2021 Western North America Heat Wave',
   'start_date': datetime.date(2021, 6, 24),
   'end_date': datetime.date(2021, 7, 2),
   'location': {'latitude': 47.6062, 'longitude': -122.3321},
   'bounding_box_km': 500,
   'cross_listed': None}]}

In [17]:
events_data['heat_wave']

[{'id': 1,
  'title': '2021 Western North America Heat Wave',
  'start_date': datetime.date(2021, 6, 24),
  'end_date': datetime.date(2021, 7, 2),
  'location': {'latitude': 47.6062, 'longitude': -122.3321},
  'bounding_box_km': 500,
  'cross_listed': None}]

In [19]:
events_data['heat_wave'][0]

{'id': 1,
 'title': '2021 Western North America Heat Wave',
 'start_date': datetime.date(2021, 6, 24),
 'end_date': datetime.date(2021, 7, 2),
 'location': {'latitude': 47.6062, 'longitude': -122.3321},
 'bounding_box_km': 500,
 'cross_listed': None}

In [18]:
dacite.from_dict(data_class=CaseTest, data=events_data['heat_wave'][0])

WrongTypeError: wrong value type for field "start_date" - should be "str" instead of value "2021-06-24" of type "date"

In [69]:
import os
import yaml
import dacite
import dataclasses
import datetime
from collections import namedtuple

Location = namedtuple('Location', ['latitude', 'longitude'])

@dataclasses.dataclass
class IndividualCase:
    """
    IndividualCase holds the metadata for a single case
    based on the events.yaml metadata.
    Attributes:
        id: the numerical identifier for the event
        start_date: datetime.date: the start date of the case
        end_date: datetime.date: the end date of the case
        location: Location: the latitude and longitude of the center of the location
        bounding_box_km: int: the side length of the square in kilometers
    """

    id: int
    start_date: datetime.date
    end_date: datetime.date
    location: Location
    bounding_box_km: int


@dataclasses.dataclass
class Event:
    """
    Event holds the cases for a specific event type.
    Attributes:
        event: list[IndividualCase]: the list of cases for the event
    """

    @classmethod
    def from_yaml(cls, path):
        with open(path, 'r') as file:
            case_dict = yaml.safe_load(file)['events']
            for case in case_dict:
                for event in case_dict[case]:
                    event['location'] = Location(**event['location'])
        return dacite.from_dict(cls, case_dict)   

@dataclasses.dataclass
class HeatWave(Event):
    """
    HeatWave holds the cases for extreme heat wave events.
    Attributes:
        heat_wave: list[IndividualCase]: the list of cases for the event
    """
    heat_wave: list[IndividualCase]


@dataclasses.dataclass
class Freeze(Event):
    """
    Freeze holds the cases for extreme freeze events.
    Attributes:
        freeze: list[IndividualCase]: the list of cases for the event
    """
    freeze: list[IndividualCase]



events_file_path  = 'assets/data/events.yaml'
heat_cases = HeatWave.from_yaml(events_file_path)
freeze_case = Freeze.from_yaml(events_file_path)

In [73]:
heat_cases.heat_wave

[IndividualCase(id=1, start_date=datetime.date(2021, 6, 24), end_date=datetime.date(2021, 7, 2), location=Location(latitude=47.6062, longitude=-122.3321), bounding_box_km=500),
 IndividualCase(id=3, start_date=datetime.date(2021, 6, 24), end_date=datetime.date(2021, 7, 2), location=Location(latitude=34.0522, longitude=-118.2437), bounding_box_km=500)]